In [1]:
import pandas as pd
from datetime import date, timedelta
import yfinance as yf

In [2]:
# read csv
df = pd.read_csv('transactions_sample.csv')
print(df)

          date        type      category              asset ticker  quantity  \
0   2026-01-05      income        salary  Monthly Allowance    NaN         0   
1   2026-01-08     expense          food              Lunch    NaN         0   
2   2026-01-10     expense     transport                MRT    NaN         0   
3   2026-01-12  investment    investment          Apple Inc   AAPL         2   
4   2026-01-15  investment    investment        NVIDIA Corp   NVDA         3   
5   2026-02-01      income        salary  Monthly Allowance    NaN         0   
6   2026-02-06     expense  subscription            Spotify    NaN         0   
7   2026-02-10  investment    investment          Apple Inc   AAPL         1   
8   2026-02-18  investment    investment        NVIDIA Corp   NVDA        -1   
9   2026-03-01      income        salary  Monthly Allowance    NaN         0   
10  2026-03-04     expense          food             Dinner    NaN         0   
11  2026-03-10  investment    investment

In [3]:
# data validation

# convert date to datetime
df["date"] = pd.to_datetime(df["date"])

# check missing values
print(df.isnull().sum())

# check invalid buys
invalid_buys = df[
    (df["type"] == "investment") &
    (df["quantity"] > 0) &
    (df["amount"] > 0)
]

print(invalid_buys)

#check amount consistency

investment_rows = df[df["type"] == "investment"]
calculated = investment_rows["quantity"] * investment_rows["price"]
print(investment_rows["amount"] + calculated)

date        0
type        0
category    0
asset       0
ticker      8
quantity    0
price       0
amount      0
account     0
notes       0
dtype: int64
Empty DataFrame
Columns: [date, type, category, asset, ticker, quantity, price, amount, account, notes]
Index: []
3     0.0
4     0.0
7     0.0
8     0.0
11    0.0
dtype: float64


In [4]:
#functions

def get_investment_transactions(df):
    investments = df[df["type"] == 'investment']
    return investments

def get_expense(df):
    expenses = df[df["type"] == 'expense']
    return expenses

def get_income(df):
    income = df[df["type"] == 'income']
    return income
print(get_income(df)["amount"].sum())

def filter_by_date(df, start, end):
    filtered_df = df[
        (df["date"] >= start) &
        (df["date"] <= end)
    ]
    return filtered_df

# function calls
print(get_investment_transactions(df)["amount"].sum())
print(get_expense(df)["amount"].sum())
print(get_income(df)["amount"].sum())
print(filter_by_date(df, '2026-01-12', '2026-03-01'))

3600.0
-1010.0
-198.68
3600.0
        date        type      category              asset ticker  quantity  \
3 2026-01-12  investment    investment          Apple Inc   AAPL         2   
4 2026-01-15  investment    investment        NVIDIA Corp   NVDA         3   
5 2026-02-01      income        salary  Monthly Allowance    NaN         0   
6 2026-02-06     expense  subscription            Spotify    NaN         0   
7 2026-02-10  investment    investment          Apple Inc   AAPL         1   
8 2026-02-18  investment    investment        NVIDIA Corp   NVDA        -1   
9 2026-03-01      income        salary  Monthly Allowance    NaN         0   

   price   amount         account                 notes  
3  180.0  -360.00       brokerage      Initial AAPL buy  
4  150.0  -450.00       brokerage      Initial NVDA buy  
5    0.0  1200.00  bank-spendable    February allowance  
6    0.0    -9.98  bank-spendable  Monthly subscription  
7  175.0  -175.00       brokerage           Add to AAPL

In [5]:
investment_df = get_investment_transactions(df)[["asset","ticker","price","quantity","amount"]]
print(investment_df.to_string(index=False))

# validate investments 
def validate(df):
    invalid_investments = df[
        (df["price"] * df["quantity"] != abs(df["amount"])) |
        (df["price"] <= 0)
    ]
    return invalid_investments
print(validate(investment_df))

      asset ticker  price  quantity  amount
  Apple Inc   AAPL  180.0         2  -360.0
NVIDIA Corp   NVDA  150.0         3  -450.0
  Apple Inc   AAPL  175.0         1  -175.0
NVIDIA Corp   NVDA  165.0        -1   165.0
  Apple Inc   AAPL  190.0         1  -190.0
         asset ticker  price  quantity  amount
8  NVIDIA Corp   NVDA  165.0        -1   165.0


In [6]:
# holdings
investment_df = investment_df.drop(columns=["price"])
group = investment_df.groupby("ticker").agg({
    "asset": "first",
    "quantity": "sum",
    "amount": "sum"
})

#group = group.reset_index()
print(group)

              asset  quantity  amount
ticker                               
AAPL      Apple Inc         4  -725.0
NVDA    NVIDIA Corp         2  -285.0


In [7]:
# validate quantity cannot be negative 
def check_quantity(df):
    invalid_investments = df[
        df["quantity"] < 0
    ]
    return invalid_investments

print(check_quantity(group)) # return empty dataframe if no negative quantity

Empty DataFrame
Columns: [asset, quantity, amount]
Index: []


In [8]:
# handle scenario when quantity is zero 
holdings = group[group["quantity"] > 0].copy()

holdings["avg_price"] = abs(holdings["amount"] / holdings["quantity"])
holdings["net_cash_flow"] = abs(holdings["amount"])
holdings = holdings.drop(columns=["amount"])

# capital invested
capital_invested = investment_df[investment_df["amount"] < 0].groupby("ticker").sum()
holdings["capital_invested"] = abs(capital_invested["amount"])
print(holdings)
print(capital_invested)

              asset  quantity  avg_price  net_cash_flow  capital_invested
ticker                                                                   
AAPL      Apple Inc         4     181.25          725.0             725.0
NVDA    NVIDIA Corp         2     142.50          285.0             450.0
                              asset  quantity  amount
ticker                                               
AAPL    Apple IncApple IncApple Inc         4  -725.0
NVDA                    NVIDIA Corp         3  -450.0


In [37]:
# fetch live prices
def fetch_live_prices(tickers):
    ticker_objects = []
    for ticker in tickers:
        ticker_object = yf.Ticker(ticker)
        ticker_objects += [ticker_object]
    live_prices = []
    for obj in ticker_objects:
        price = round(obj.fast_info['last_price'],2)
        live_prices += [price]
    
    return live_prices

In [39]:
investment_copy = investment_df.copy()
group_copy = investment_df.groupby("ticker").sum()
group_copy = group_copy.reset_index()

series = group_copy["ticker"]

tickers = group_copy["ticker"].to_list()

In [40]:
# current price -- temporary for now (will use live tracking APIs eventually)
holdings["current_price"] = fetch_live_prices(tickers)

# calculate market value
holdings["market_value"] = holdings["quantity"] * holdings["current_price"]
print(holdings)

              asset  quantity  avg_price  net_cash_flow  capital_invested  \
ticker                                                                      
AAPL      Apple Inc         4     181.25          725.0             725.0   
NVDA    NVIDIA Corp         2     142.50          285.0             450.0   

        current_price  market_value  
ticker                               
AAPL           271.86       1087.44  
NVDA           186.50        373.00  


In [41]:
# unrealised P/L
holdings["unrealised_pnl"] = (holdings["current_price"] - holdings["avg_price"]) * holdings["quantity"]

# unrealised P/L %
holdings["unrealised_pnl_pct"] = round((holdings["unrealised_pnl"] / holdings["capital_invested"]) * 100, 2)

# display
display_df = holdings.copy()
display_df["unrealised_pnl_pct"] = display_df["unrealised_pnl_pct"].map("{:.2f}%".format)
print(display_df)

              asset  quantity  avg_price  net_cash_flow  capital_invested  \
ticker                                                                      
AAPL      Apple Inc         4     181.25          725.0             725.0   
NVDA    NVIDIA Corp         2     142.50          285.0             450.0   

        current_price  market_value  unrealised_pnl unrealised_pnl_pct  
ticker                                                                  
AAPL           271.86       1087.44          362.44             49.99%  
NVDA           186.50        373.00           88.00             19.56%  


In [ ]:
# question: is net exposure correct naming
# add percentage symbol
# group holdings by ticker but keep it consistent format
# is it smart to drop price after as it requires sequential running of cells 

# raw_df --> calculations
# holdings_df --> analytics
# display_df --> formatting only

In [42]:
# compute total capital invested
total_capital_invested = holdings["capital_invested"].sum()
print(total_capital_invested)

# compute total market value
total_market_value = holdings["market_value"].sum()
print(total_market_value)

#compute total unrealised p/l
total_unrealised_pnl = holdings["unrealised_pnl"].sum()
print(total_unrealised_pnl)

# compute portfolio-level p/l%
portfolio_pnl_pct = round((total_unrealised_pnl / total_capital_invested) * 100,2)
print(portfolio_pnl_pct) # pct symbol later

1175.0
1460.44
450.44000000000005
38.34


In [43]:
# calculate weight (%) of each holding
holdings["weight"] = round((holdings["market_value"] / total_market_value) * 100,2)

# validate weights sums to 100%
def check_weights(df):
    total_weight =  df["weight"].sum()
    return total_weight

print(check_weights(holdings)) # returns 100 if correct


# display
display_df = holdings.copy()
display_df["unrealised_pnl_pct"] = display_df["unrealised_pnl_pct"].map("{:.2f}%".format)
display_df["weight"] = display_df["weight"].map("{:.2f}%".format)

# sort holdings by weight descending 
display_df.sort_values(by='weight', ascending=False) 

100.0


,asset,quantity,avg_price,net_cash_flow,capital_invested,current_price,market_value,unrealised_pnl,unrealised_pnl_pct,weight
ticker,,,,,,,,,,
AAPL,Apple Inc,4,181.25,725.0,725.0,271.86,1087.44,362.44,49.99%,74.46%
NVDA,NVIDIA Corp,2,142.50,285.0,450.0,186.50,373.00,88.00,19.56%,25.54%


In [44]:
portfolio = {
    "total_capital_invested": total_capital_invested,
    "total_market_value": total_market_value,
    "total_unrealised_pnl": total_unrealised_pnl,
    "portfolio_pnl_pct": portfolio_pnl_pct
}

# portfolio validations (holdings)
#next time